# Outbound Auth

Outbound Auth는 Agent와 AgentCore Gateway가 Inbound Auth 중에 인증 및 권한이 부여된 사용자를 대신하여 AWS 리소스 및 타사 서비스에 안전하게 액세스할 수 있도록 합니다. AWS 리소스 또는 타사 서비스와 권한 부여를 통합하려면 Inbound Auth와 Outbound Auth를 모두 구성해야 합니다.

AgentCore Identity가 지원하는 최소 권한 액세스 및 안전한 권한 위임을 통해 Agent는 GitHub, Google, Salesforce, Slack과 같은 AWS 리소스 및 타사 도구에 원활하고 안전하게 액세스할 수 있습니다. Agent는 사전 승인된 사용자 동의가 있는 경우 사용자를 대신하거나 독립적으로 이러한 서비스에서 작업을 수행할 수 있습니다. 또한 안전한 토큰 볼트를 사용하여 동의 피로를 줄이고 간소화된 AI Agent 경험을 만들 수 있습니다.

## Outbound Authentication 구성

먼저 타사 Provider에 클라이언트 애플리케이션을 등록한 다음 Outbound Auth를 생성합니다. AWS 리소스 또는 타사 서비스 또는 AgentCore Gateway 대상에 대한 액세스를 검증하는 방법을 지정합니다. OAuth 2LO/3LO 또는 API 키를 사용할 수 있습니다. OAuth를 사용하면 AgentCore Identity가 제공하는 Provider 중에서 선택할 수 있습니다. 이 경우 AgentCore Identity에서 Provider에 대한 구성 세부 정보를 입력합니다. 또는 커스텀 Provider에 대한 세부 정보를 제공할 수 있습니다.

사용자가 AWS 리소스 또는 타사 서비스 또는 AgentCore Gateway 대상에 액세스하려는 경우 Outbound Auth는 Incoming Auth에서 제공한 액세스 토큰이 유효한지 확인하고 유효한 경우 리소스에 대한 액세스를 허용합니다.

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>


@require_access_token 데코레이터와 함께 사용할 수 있는 다양한 파라미터는 다음과 같습니다.


| Parameter Name      | Description                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | The credential provider name                                             |
| into                | Parameter name to inject the token into                                  |
| scopes              | OAuth2 scopes to request                                                 |
| on_auth_url	      | Callback for handling authorization URLs                                 |
| auth_flow           | Authentication flow type ("M2M" or "USER_FEDERATION")                    |
| callback_url        | OAuth2 callback URL                                                      |
| force_authentication| Force re-authentication                                                  |
| token_poller        | Custom token poller implementation                                       |

		


# Amazon Bedrock AgentCore Runtime에서 Strands Agent 호스팅

## 개요


이 튜토리얼에서는 사용자의 Google Calendar에서 이벤트를 나열할 수 있는 Strands Agent를 사용하여 일정 관리 Agent를 개발합니다. Google과의 자격 증명 관리를 돕기 위해 credential provider를 구성합니다. Google용 명명된 Provider를 사용하고 Agent 코드를 수정하여 credential provider를 호출하고 access_token을 사용하여 Google에서 사용자의 캘린더 이벤트 또는 일정을 가져올 수 있습니다.

### 튜토리얼 아키텍처

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### 튜토리얼 세부 정보

| Information         | Details                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                           |
| Agent type          | Single                                                                   |
| Agentic Framework   | Strands Agents                                                           |
| LLM model           | Anthropic Claude Haiku 4.5                                              |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Claude Model |
| Tutorial vertical   | Cross-vertical                                                           |
| Example complexity  | Medium                                                                   |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                             |
| Credential Provider | Type : OAuth2 - Google Provider                                          |


### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore Runtime에서 Agent 호스팅
* Claude Model 사용
* Strands Agent 사용
* OAuth2 Google credential provider와 함께 AgentCore egress Auth 사용


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

##  IDP로 Cognito를 사용하여 Inbound Auth 구성
App client와 하나의 테스트 사용자가 있는 Cognito Userpool을 프로비저닝하겠습니다. 배포된 MCP 서버에 액세스하기 위한 JWT 토큰을 제공하기 위해 Amazon Cognito를 사용합니다. 이를 위해 `utils` 스크립트의 `setup_cognito_user_pool` 지원 함수를 사용합니다.

참고: Cognito access_token은 2시간 동안만 유효합니다. access_token이 만료되면 `reauthenticate_user` 메서드를 사용하여 다른 access_token을 발급할 수 있습니다.

In [ ]:
import sys
import os
import subprocess
from boto3.session import Session
from utils import setup_cognito_user_pool, reauthenticate_user

# 현재 노트북의 디렉토리 경로 가져오기
current_dir = os.path.dirname(
    os.path.abspath("__file__" if "__file__" in globals() else ".")
)

# utils 디렉토리 경로 설정 (상위 디렉토리)
utils_dir = os.path.join(current_dir, "..")
utils_dir = os.path.abspath(utils_dir)

# sys.path에 utils 디렉토리 추가하여 모듈 import 가능하게 함
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

# AWS 세션 및 리전 설정
boto_session = Session()
region = boto_session.region_name

print(f"Region: {region}")

# AgentCore Identity 클라이언트 생성
identity_client = boto_session.client("bedrock-agentcore-control")

In [ ]:
region

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool("Cognito_3LO_Google")
print("Cognito setup completed ✓")

## OAuth2용 Google 구성 ( 사용자 대신 플로우 )
다음 단계에 따라 앱을 등록하고, 프로젝트를 생성하고, Google Calendar 액세스를 위한 OAuth 자격 증명을 구성하세요:

이 섹션에서는 읽기 전용 권한으로 Google Calendar API에 액세스하도록 Google을 구성합니다.

1. Google Developer Console에서 프로젝트 생성
    1.    Go to the [Google Developer Console](https://console.developers.google.com/)
    2.    In the top navigation bar, click on “Create Project”
    3.    Enter a Project Name.
    4.    Choose an Organization or leave as “No organization” if not applicable.
    5.    Click Create. Your new project will appear in the project list.
2. Google Calendar API 활성화
    1.    With your project selected ( using the checkbox),  open the left menu (hamburger menu) and go to APIs & Services > Library.
    2.    In the search bar, type Google Calendar API.
    3.    Click on Google Calendar API in the results, then click Enable.
3. OAuth Consent Screen 구성
    1.    In the left menu, go to APIs & Services > OAuth consent screen.
    2.    Click “Get started”
    3.    Fill in the required fields:
    4.    App Name
    5.    User Support Email
    6. Click Next and then select the right Audience type, i.e. Internal or External, Click Next, ( If you select External, then ensure you have entered email ids of users entered here, so you can test with those users)
    7.    Developer Contact Information, Enter your email id.
    8.  Click “Finish” after accepting the terms and conditions. an then click “Create”.
4. 테스트 사용자로 Google 이메일 ID 추가
    1.    In the left menu, go to APIs & Services > OAuth consent screen.
    2.    Select "Audience" from the left hand menu.
    3.    Under Test users, click "+ Add Users" and add your gmail id for the Google account.
5. OAuth 2.0 Credentials 생성
    1.    Go to APIs & Services from the left hand menu and then select  > Credentials.
    2.    Click Create Credentials > OAuth client ID.
    3.    Select Web application as the application type.
    4.    Enter a name for the credentials.
    5.    Click Create.
6. Client ID 및 Client Secret 얻기
    1.    After creation, a dialog will display your Client ID and Client Secret. Copy these for later use.
    2.    Download the credentials as a JSON file or copy them for use in your app’s configuration.
7. Data access 업데이트 
    1. Go to APIs & Services from the left hand menu and then select  > Credentials.
    2. Select the “web app” you created in step 4d.
    3. Select “Data access” on the left hand menu
    4. Click “Add or remove scopes”
    5. Add scopes based on your use case. E.g: For Google calendar add, “https://www.googleapis.com/auth/calendar.readonly” under “Manually add scopes” and click update followed by Save/Update.
    6. Click “Save” again on the “Data access” page to save your configuration.
8. Agent에서 Credentials 사용
    1.    In the next section, we will configure a resource credentials provider to use the Client ID, Client Secret, and Redirect URI for the OAuth 3-legged flow.

## OAuth2 Authorization URL Session Binding 프로세스

OAuth2 authorization URL session binding 프로세스는 OAuth2 권한 부여 세션이 AgentCore Identity의 인증된 사용자와 올바르게 연결되도록 하는 중요한 보안 메커니즘입니다. 이 프로세스는 세션 하이재킹을 방지하고 OAuth 토큰이 의도된 사용자에게만 부여되도록 합니다.

Ref : https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html

### Session binding 작동 방식
<div style="text-align:center">
    <img src="images/identity-session-binding.png" width="90%"/>
</div>

1. Agent 호출 – 원래 Agent 사용자가 자신이 소유한 일부 애플리케이션 또는 리소스에 액세스하려는 경우 Agent 코드가 GetResourceOauth2Token API를 호출하여 권한 부여 URL을 검색합니다.

2. 권한 부여 URL 생성 – AgentCore Identity는 사용자가 탐색하고 액세스에 동의할 수 있도록 권한 부여 URL 및 세션 URI를 생성합니다.

3. 권한 부여 및 액세스 토큰 얻기 – 사용자가 권한 부여 URL로 이동하여 Agent가 자신의 리소스에 액세스하도록 동의합니다. 그 후 AgentCore Identity는 권한 부여 요청의 원래 사용자를 포함하는 정보와 함께 사용자의 브라우저를 HTTPS 애플리케이션 엔드포인트로 리디렉션합니다. 이 시점에서 HTTPS 애플리케이션 엔드포인트는 원래 Agent 사용자가 애플리케이션의 현재 로그인한 사용자와 동일한지 확인합니다. 일치하면 애플리케이션 엔드포인트가 CompleteResourceTokenAuth를 호출하여 AgentCore Identity가 액세스 토큰을 가져와 저장할 수 있도록 합니다.

4. 액세스 토큰을 얻기 위해 Agent 재호출 – 애플리케이션이 유효한 응답을 반환하면 Agent 애플리케이션은 원래 사용자에 대해 요청된 OAuth2.0 액세스 토큰을 검색할 수 있습니다. 사용자가 일치하지 않으면 애플리케이션은 아무 작업도 수행하지 않거나 시도를 기록합니다.

애플리케이션 엔드포인트가 사용자 ID를 확인할 수 있도록 함으로써 AgentCore Identity는 Agent 애플리케이션이 권한 부여 요청을 시작한 사용자와 액세스에 동의한 사용자가 항상 동일한 사용자임을 보장할 수 있도록 합니다.

### 이 샘플의 OAuth2 Session Binding 플로우 개요

OAuth2 session binding 프로세스에는 애플리케이션, AgentCore Identity, 외부 OAuth Provider(Google, Github 등) 및 로컬 콜백 서버 간에 조정되는 몇 가지 주요 단계가 포함됩니다:

#### 1단계: 애플리케이션 Callback URL 생성
- 애플리케이션에서 공개적으로 액세스 가능한 HTTPS 콜백 엔드포인트 생성
- 이 엔드포인트는 OAuth 리디렉션을 처리하고 사용자 세션을 검증함
- Example: `https://myagentapp.com/callback`

#### 2단계: Callback URL로 Workload Identity 업데이트
- workload identity에서 콜백 URL을 `AllowedResourceOauth2ReturnUrl`로 등록
- 이는 `UpdateWorkloadIdentity` API를 사용하여 수행됨
- **이 튜토리얼에서**: 이 단계는 로컬 콜백 서버 URL로 workload identity를 업데이트하는 아래 코드에 의해 자동으로 처리됨

#### 3단계: OAuth2 Credential Provider 생성
- 외부 OAuth Provider 세부 정보(client ID, secret)로 credential provider 구성
- AgentCore Identity는 Provider에 대한 고유한 콜백 URL을 반환함
- 외부 OAuth Provider(예: Google Console)에 이 콜백 URL 등록

#### 4단계: 세션 검증 및 토큰 완료 구현
- 콜백 엔드포인트는 현재 사용자의 세션을 검증해야 함
- 사용자 식별자 및 세션 URI와 함께 `CompleteResourceTokenAuth` API 호출
- **이 튜토리얼에서**: `oauth2_callback_server.py`가 이를 자동으로 처리함

#### 5단계: OAuth 플로우 실행
- 사용자가 Agent 상호 작용을 통해 OAuth 플로우를 트리거함
- 사용자가 권한 부여를 위해 외부 Provider로 리디렉션됨
- Provider가 세션 정보와 함께 콜백으로 다시 리디렉션함
- 세션 바인딩이 완료되고 OAuth 토큰을 사용할 수 있게 됨

### oauth2_callback_server.py를 사용한 로컬 개발

로컬 개발 및 테스트를 위해 이 튜토리얼은 다음을 수행하는 `oauth2_callback_server.py`를 사용합니다:

1. **로컬 FastAPI 서버 실행** (`localhost:9090`)
   - 상태 확인을 위한 `/ping` 엔드포인트 제공
   - 사용자 토큰을 저장하기 위한 `/userIdentifier/token` 엔드포인트 제공
   - OAuth 리디렉션을 처리하기 위한 `/oauth2/callback` 엔드포인트 제공

2. **사용자 토큰 저장소 관리**
   - Cognito 인증의 사용자 JWT 토큰 저장
   - OAuth 세션을 올바른 사용자 ID와 연결

3. **OAuth 콜백 처리**
   - `session_id` 파라미터와 함께 OAuth 리디렉션 수신
   - 세션을 바인딩하기 위해 `CompleteResourceTokenAuth` 호출
   - 플로우를 완료하기 전에 사용자 ID 검증

4. **세션 보안 제공**
   - OAuth 세션이 인증된 사용자에게 바인딩되도록 보장
   - OAuth 토큰에 대한 무단 액세스 방지

### Workload Identity 업데이트와의 통합

참조한 코드 스니펫은 OAuth2 세션 바인딩 프로세스에서 중요한 단계를 수행합니다:

```python
if launch_result.agent_id:
    workload_name = launch_result.agent_id
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
```

이 코드는:
1. **Agent의 Workload Identity 검색**: 런타임 배포의 Agent ID 사용
2. **현재 허용된 URL 가져오기**: 기존 `allowedResourceOauth2ReturnUrls` 가져오기
3. **로컬 Callback URL 추가**: 허용된 반환 URL로 `http://localhost:9090/oauth2/callback` 포함
4. **Workload Identity 업데이트**: AgentCore Identity에 콜백 URL 등록

이 등록은 AgentCore Identity가 사전 등록된 URL로만 OAuth 콜백을 리디렉션하여 추가 보안 계층을 제공하기 때문에 필수적입니다.

### 보안 고려 사항

OAuth2 세션 바인딩 프로세스에는 여러 보안 조치가 포함됩니다:
- **URL 검증**: 사전 등록된 콜백 URL만 허용됨
- **세션 검증**: 토큰 완료 전에 사용자 세션을 검증해야 함
- **사용자 ID 바인딩**: OAuth 세션이 인증된 사용자에게 명시적으로 바인딩됨
- **토큰 격리**: 각 사용자의 OAuth 토큰이 격리되고 안전함

이 포괄적인 접근 방식은 멀티 사용자 환경에서 OAuth2 플로우가 안전하고 올바른 사용자에게 적절하게 귀속되도록 보장합니다.

---

### Google OAuth2 credential provider 구성

현재 폴더에 `.env` 파일을 생성하고 다음 텍스트를 복사합니다:
```sh
GOOGLE_CLIENT_ID="" #"client id" recorded from Step 6.1 above
GOOGLE_CLIENT_SECRET="" #"client secret" recorded from Step 6.2 above.
```

client id와 client secret이 업데이트되면 아래 코드를 실행하여 Google용 credentials provider를 생성합니다. <br>
AgentCore Identity의 Resource credential provider는 Agent, identity provider 및 리소스 서버 간의 복잡한 관계를 관리하는 지능형 중개자 역할을 합니다. 각 Provider는 특정 서비스 또는 ID 시스템에 필요한 특정 엔드포인트 구성을 캡슐화합니다. 이 서비스는 개발 노력을 줄이기 위해 권한 부여 서버 엔드포인트 및 Provider별 파라미터가 사전 구성된 Google, GitHub, Slack 및 Salesforce를 포함한 인기 있는 서비스에 대한 기본 제공 Provider를 제공합니다. AgentCore Identity는 OAuth2 호환 리소스 서버와 함께 작동하도록 맞춤 설정할 수 있는 구성 가능한 OAuth2 credential provider를 통해 커스텀 구성을 지원합니다.


In [ ]:
%%writefile .env
GOOGLE_CLIENT_ID="" #"client id" recorded from Step 6.1 above
GOOGLE_CLIENT_SECRET="" #"client secret" recorded from Step 6.2 above.

In [ ]:
import dotenv

dotenv.load_dotenv(override=True)

# Google OAuth2 credential provider 생성 (3-legged OAuth 플로우)
google_provider = identity_client.create_oauth2_credential_provider(
    **{
        "name": "google-cal-provider",
        "credentialProviderVendor": "GoogleOauth2",  # Google OAuth2 provider 사용
        "oauth2ProviderConfigInput": {
            "googleOauth2ProviderConfig": {
                "clientId": os.environ["GOOGLE_CLIENT_ID"],  # .env 파일에서 client ID 로드
                "clientSecret": os.environ["GOOGLE_CLIENT_SECRET"],  # .env 파일에서 client secret 로드
            }
        },
    }
)
print(google_provider)
print("\n")
print(f"callbackUrl: {google_provider['callbackUrl']}")

## Google/OAuth 2.0 클라이언트에서 callback url 업데이트 
[Google Developer Console](https://console.developers.google.com/)로 다시 이동

1. Google Developer Console에서 프로젝트 선택
    1.    이전에 생성한 프로젝트 선택
2. callback uri 업데이트
    1.    Go to APIs & Services from the left hand menu and then select  > Credentials.
    2.    "OAuth 2.0 Client IDs" 아래의 클라이언트 클릭
    3.    "Authorised redirect URIs" 아래에 이전 단계의 callback url 입력. callback url이 출력되었으므로 이전 단계에서 쉽게 복사할 수 있음
    4.    Save 클릭

## AgentCore Runtime에 배포하기 위한 Agent 준비

### Amazon Bedrock에서 호스팅되는 Model을 사용하는 Strands Agent
다음을 포함하는 Strands Agent 코드입니다:
1. 오늘의 Google Calendar에서 이벤트를 가져오기 위해 "get_calendar_events_today"라는 새 도구 생성
2. 이전 단계에서 생성한 Credential provider를 사용하여 Google에서 access_token을 가져옴. 여기에는 3LO 플로우의 일부로 사용자에게 승인을 위해 동의가 전송되는 사용자 동의 플로우가 포함됨
3. Strands Agent는 사용자 일정과 관련된 모든 사용자 요청에 대해 도구를 호출함

In [ ]:
# 현재 환경(notebook/SageMaker)에 따라 OAuth2 callback URL 가져오기
# 이 코드는 agent 컨테이너가 아닌 노트북에서 실행됨
from oauth2_callback_server import get_oauth2_callback_url

oauth2_callback_url_for_agent = get_oauth2_callback_url()

print(
    f"Callback URL for agent (determined from notebook environment): {oauth2_callback_url_for_agent}"
)

## AgentCore Runtime에 Agent 배포
CreateAgentRuntime 작업은 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있는 포괄적인 구성 옵션을 지원합니다. 또한 클라이언트가 Agent와 통신하는 방법을 제어하기 위해 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성할 수 있습니다.

참고: 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR로 푸시하는 것입니다

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore 런타임에 배포합니다.


### AgentCore Runtime 배포 구성

다음으로 스타터 툴킷을 사용하여 엔트리포인트, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime 배포를 구성합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다

참고: authorizer_configuration은 Cognito를 사용한 Inbound Auth용으로 구성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

print(f"Region: {region}")

# Cognito 설정에서 discovery URL과 client ID 가져오기
discovery_url = cognito_config.get("discovery_url", "")
client_id = cognito_config.get("client_id", "")
agentcore_runtime = Runtime()

# AgentCore Runtime 구성 설정
response = agentcore_runtime.configure(
    entrypoint="strands_claude_google_3lo.py",  # Agent 진입점 파일
    auto_create_execution_role=True,  # 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR 리포지토리 자동 생성
    requirements_file="requirements.txt",
    region=region,
    memory_mode="NO_MEMORY",  # 메모리 모드 설정
    agent_name="strands_agent_google_3lo",
    authorizer_configuration={  # Cognito JWT 인증 설정
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id],
        }
    },
)
print(response)

## AgentCore 구성 검토

In [ ]:
!cat .bedrock_agentcore.yaml

### AgentCore Runtime에 Agent 시작

이제 docker 파일이 있으므로 AgentCore Runtime에 Agent를 시작하겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
from oauth2_callback_server import get_oauth2_callback_url

# Agent를 AgentCore Runtime에 배포하고 배포 세부 정보 가져오기
launch_result = agentcore_runtime.launch(
    env_vars={"CALLBACK_URL": oauth2_callback_url_for_agent},  # 환경 변수로 callback URL 전달
    auto_update_on_conflict=True,  # 충돌 시 자동 업데이트
)
print(launch_result)

if launch_result.agent_id:
    # 배포된 agent의 ID에서 workload name 추출 (identity 관리용)
    workload_name = launch_result.agent_id
    # AgentCore Identity에서 현재 workload identity 구성 가져오기
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    # 이 workload에 이미 등록된 OAuth2 callback URL 추출
    allowed_resource_oauth_2_return_urls = (
        workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    )
    # 세션 바인딩을 위한 로컬 OAuth2 callback 서버 URL 가져오기 (localhost:9090/oauth2/callback)
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    # OAuth2 세션 바인딩을 활성화하기 위해 workload identity에 로컬 callback URL 등록
    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowedResourceOauth2ReturnUrls=[
            *allowed_resource_oauth_2_return_urls,
            oauth2_callback_url,
        ],
    )
    print(updated_workload_identity)

#### 자동 생성된 역할에 추가 필수 정책 추가

이전에 Model에 액세스하지 않은 새 계정에서 이를 사용하는 경우 Agent가 Model에 액세스할 수 있도록 자동 생성된 역할에 추가 필수 정책을 추가해야 합니다.

In [ ]:
import json
import boto3

agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

# Agent runtime 정보 가져오기
runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response["roleArn"]
account = boto_session.client("sts").get_caller_identity().get("Account")

# Runtime 역할에 추가할 정책 정의
policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockModelAccess",
            "Effect": "Allow",
            "Action": [
                "aws-marketplace:ViewSubscriptions",
                "aws-marketplace:Subscribe",
            ],
            "Resource": "*",
        },
        {
            "Sid": "Oauth2TokenAccess",  # OAuth2 토큰 가져오기 권한
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceOauth2Token",
            ],
            "Resource": "*",
        },
        {
            "Sid": "SecretsManagerAccess",  # Secrets Manager에서 Google OAuth2 credential 읽기 권한
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue",
            ],
            "Resource": [
                f"arn:aws:secretsmanager:{region}:{account}:secret:secret:bedrock-agentcore-identity!default/oauth2/google-cal-provider*"
            ],
        },
    ],
}
iam_client = boto3.client("iam", region_name=region)

# IAM 역할에 정책 추가
response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인하겠습니다

In [ ]:
import time

# AgentCore Runtime 상태 확인
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
# 배포가 완료될 때까지 대기 (10초마다 상태 확인)
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)
print(f"Final status: {status}")

### AgentCore Runtime 호출

마지막으로 페이로드로 AgentCore Runtime을 호출할 수 있습니다

Agent가 "Get_calendar_events_today" 도구를 호출하고 3 Legged OAuth 플로우를 트리거하는 것을 확인할 수 있습니다. Authorization Url이 표시됩니다. Authorization url을 클릭하거나 새 브라우저 세션/탭에 복사/붙여넣기하여 사용자 동의 플로우를 완료하세요.
권한 부여가 완료되면 credential provider "google-cal-provider"가 Google에서 access_token을 가져와 캘린더에서 이벤트를 가져오는 도구 실행을 완료합니다.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
from oauth2_callback_server import (
    store_token_in_oauth2_callback_server,
    wait_for_oauth2_server_to_be_ready,
)

# Cognito에서 새로운 bearer token 발급
bearer_token = reauthenticate_user(cognito_config.get("client_id"))

# OAuth2 callback 서버 시작 명령어 구성
oauth2_callback_server_cmd = [
    sys.executable,
    "oauth2_callback_server.py",
    "--region",
    region,
]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    # OAuth2 callback 서버가 준비될 때까지 대기
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print(
            "Failed to start OAuth2 callback server to handle session binding "
            "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)"
        )
    else:
        # OAuth2 callback 서버에 bearer token 저장 (세션 바인딩용)
        store_token_in_oauth2_callback_server(bearer_token)
        # Agent 호출 (Google Calendar 이벤트 조회)
        invoke_response = agentcore_runtime.invoke(
            {"prompt": "What is in my agenda for today? Highlight the main events!"},
            bearer_token=bearer_token,
        )
        print(invoke_response)
finally:
    # OAuth2 callback 서버 종료
    oauth2_callback_server_process.terminate()

## 선택 사항 - Streamlit App을 사용하여 Agent 테스트

멋진 채팅 인터페이스를 제공하는 Streamlit 웹 애플리케이션을 사용하여 배포된 Agent를 테스트할 수 있습니다. 이 디렉토리의 `chatbot_app_cognito.py` 파일은 다음을 수행하는 웹 기반 챗봇을 생성합니다:

- `.bedrock_agentcore.yaml`에서 구성을 자동으로 읽음
- Cognito 인증 제공
- 스트리밍 응답이 있는 최신 채팅 인터페이스 표시
- Google Calendar 액세스를 위한 3LO OAuth 플로우 처리

### Streamlit App 실행

여러 가지 방법으로 Streamlit 앱을 실행할 수 있습니다:

#### 옵션 1: Jupyter Notebook에서 실행 (현재 디렉토리)
- 아래 셀을 실행하여 이 노트북에서 직접 Streamlit 앱을 시작합니다:
- 로그인: testuser / MyPassword123! 자격 증명 사용 (Cognito 설정으로 생성된 기본 테스트 사용자)
- "Tell me a joke"와 같은 간단한 프롬프트 테스트
- "What is on my agenda for today?"와 같이 `Get_calendar_events_today` 도구를 트리거하는 프롬프트로 테스트
- Authorization url이 반환되는 것을 볼 수 있습니다. url을 클릭하거나 새 브라우저 탭/창에 url을 복사/붙여넣기하여 사용자 동의 플로우를 완료하세요.

In [ ]:
from chatbot_app_cognito import get_streamlit_url

# 현재 디렉토리로 변경 (chatbot_app_cognito.py 파일이 있는 위치)
notebook_dir = os.getcwd()

# Streamlit 앱 시작
print("Starting Streamlit app...")

# OAuth2 callback 서버 시작 명령어 구성
oauth2_callback_server_cmd = [
    sys.executable,
    "oauth2_callback_server.py",
    "--region",
    region,
]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    # OAuth2 서버가 준비될 때까지 대기
    wait_for_oauth2_server_to_be_ready()

    # 현재 디렉토리에서 streamlit 실행
    process = subprocess.Popen(
        [
            sys.executable,
            "-m",
            "streamlit",
            "run",
            "chatbot_app_cognito.py",
            "--server.port=8501",
            "--server.showEmailPrompt=false",
        ],
        cwd=notebook_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )

    # 출력을 실시간으로 출력
    for line in iter(process.stdout.readline, ""):
        if line:
            if "8501" in line:
                print("\n🎉 Streamlit app is ready!")
                streamlit_url = get_streamlit_url()
                print(f"\n🚀 Streamlit Application URL:\n{streamlit_url}\n")
                print(
                    "⚠️ To stop the app, interrupt the kernel or press Ctrl+C in the terminal"
                )
                break

except KeyboardInterrupt:
    print("\nStreamlit app stopped.")
    oauth2_callback_server_process.terminate()
    process.terminate()
except Exception as e:
    print(f"Error starting Streamlit app: {e}")

#### 옵션 2: 터미널에서 실행

또는 터미널에서 Streamlit 앱을 실행할 수 있습니다:

```bash
# 현재 디렉토리로 이동
cd 01-tutorials/03-AgentCore-identity/05-Outbound_Auth_3lo/

# Streamlit 앱 실행
python oauth2_callback_server.py -r <region> & streamlit run chatbot_app_cognito.py
```

#### Streamlit App 사용

1. **로그인**: `testuser` / `MyPassword123!` 자격 증명 사용 (Cognito 설정으로 생성된 기본 테스트 사용자)
2. **채팅**: "What is in my agenda for today?" 또는 "Highlight main events from my agenda from today"와 같은 질문하기
3. **OAuth 플로우**: Agent가 Google Calendar 액세스가 필요한 경우 권한 부여 URL이 표시됨 - 클릭하여 OAuth 플로우 완료
4. **기능**: 앱에는 다음이 포함됨:
   - 실시간 스트리밍 응답
   - 클릭 가능한 URL
   - 최신 채팅 인터페이스
   - 컨텍스트 인식
   - 정보 메시지가 있는 오류 처리

앱은 `.bedrock_agentcore.yaml` 파일에서 모든 구성을 자동으로 읽으므로 방금 배포한 동일한 Agent 런타임을 사용합니다.

## 정리 (선택 사항)

- 이제 생성된 AgentCore Runtime을 정리하겠습니다
- 아래 셀의 주석을 제거하고 실행하세요.

In [ ]:
# launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
# agentcore_control_client = boto3.client(
#     'bedrock-agentcore-control',
#     region_name=region
# )
# ecr_client = boto3.client(
#     'ecr',
#     region_name=region

# )

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,

# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split('/')[1],
#     force=True
# )

## 축하합니다!